<a href="https://colab.research.google.com/github/Yetibi/Autoreel/blob/main/LIMPIEZA_Y_PROCESAMIENTO_ARCHIVOS_MENSUALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# === FUNCION MAESTRA PARA PROCESAR CUALQUIER MES ===
def procesar_mes(df):
    df.columns = df.columns.str.strip()

    # --- Renombrar columnas clave ---
    df = df.rename(columns={
        "SERVICIO": "CONCEPTO",
        "TOTAL PAGADO": "VALOR ($)",
        "BANCO": "BANCO VALOR PAGOS TRANSFERENCIAS",
        "ORDEN": "BANCO INGRESO EFECTIVO",
        "TRANSFERENCIA": "INGRESO POR TRANSFERENCIA"
    })

    # --- Eliminar encabezados repetidos en SERVICIOS ---
    encabezados_servicio = ["# ORDEN", "CONCEPTO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
                            "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "VALOR ($)"]

    def es_fila_encabezado(fila):
        if str(fila.get("CATEGORIA", "")).strip().upper() != "SERVICIOS":
            return False
        coincidencias = sum(
            str(fila.get(col, "")).strip().upper() == col.strip().upper()
            for col in encabezados_servicio
        )
        return coincidencias >= len(encabezados_servicio) - 1

    df = df[~df.apply(es_fila_encabezado, axis=1)].reset_index(drop=True)

    # --- Clasificar tipo de movimiento ---
    df["CATEGORIA"] = df["CATEGORIA"].astype(str).str.strip().str.upper()
    df["TIPO MOVIMIENTO"] = df.apply(
        lambda row: "EGRESO" if row["CATEGORIA"] in [
            "ENTREGA EFECTIVO DIA A", "ADELANTOS O PRESTAMOS", "GASTOS Ò PAGOS PROVEEDORES",
            "TRANSFERENCIAS", "VALOR PAGOS TRANSFERENCIAS", "COMISION"
        ] else "INGRESO" if row["CATEGORIA"] in ["BASE DÍA", "INGRESO EFECTIVO", "SERVICIOS"]
        else row.get("TIPO MOVIMIENTO", None),
        axis=1
    )

    # --- Asignar FORMA DE PAGO ---
    def detectar_forma_pago(row):
        formas = []
        if isinstance(row.get("EFECTIVO"), (int, float)) and row["EFECTIVO"] > 0:
            formas.append("EFECTIVO")
        if isinstance(row.get("BANCOLOMBIA"), (int, float)) and row["BANCOLOMBIA"] > 0:
            formas.append("BANCOLOMBIA")
        if isinstance(row.get("DATAFONO"), (int, float)) and row["DATAFONO"] > 0:
            formas.append("DATAFONO")
        return "/".join(formas) if formas else None

    df.insert(df.columns.get_loc("BANCOLOMBIA"), "FORMA DE PAGO", df.apply(detectar_forma_pago, axis=1))

    # --- Reemplazos especiales en CONCEPTO ---
    df["CONCEPTO"] = df["CONCEPTO"].replace({
        "Julian": "Comisión Administrativa",
        "Esteban": "Comisión Operativa"
    })
    df = df[~df["CONCEPTO"].isin(["COMISIÓN ADMINISTRATIVA", "COMISIÓN OPERATIVA"])]

    # --- Reubicar PERSONA / GASTOS / TRANSFERENCIAS / BASE DÍA ---
    df.loc[df["PERSONA"].notna(), "CONCEPTO"] = df.loc[df["PERSONA"].notna(), "PERSONA"]
    df.loc[df["PERSONA"].notna(), "VALOR ($)"] = df.loc[df["PERSONA"].notna(), "VALOR"]
    df.loc[df["CATEGORIA"] == "GASTOS Ò PAGOS PROVEEDORES", ["CONCEPTO", "VALOR ($)"]] = df.loc[
        df["CATEGORIA"] == "GASTOS Ò PAGOS PROVEEDORES", ["GASTOS Ò PAGOS PROVEEDORES", "VALOR"]
    ].values
    df.loc[df["CATEGORIA"] == "BASE DÍA", "VALOR ($)"] = df.loc[df["CATEGORIA"] == "BASE DÍA", "BASE DÍA"]
    df.loc[df["CATEGORIA"] == "BASE DÍA", "CONCEPTO"] = "Base día"

    # --- Filtrado de valores numéricos ---
    def es_valido(x):
        try:
            return float(x) > 0
        except:
            return False

    df = df[~((df["CATEGORIA"] == "ADELANTOS O PRESTAMOS") & ~df["VALOR ($)"].apply(es_valido))]
    df = df[~((df["CATEGORIA"] == "ENTREGA EFECTIVO DIA A") & ~df["VALOR ($)"].apply(es_valido))]
    df = df[~((df["CATEGORIA"] == "INGRESO EFECTIVO") & (~df["VALOR ($)"].apply(es_valido)))].reset_index(drop=True)

    # --- Reubicar datos de INGRESO EFECTIVO a CONCEPTO y VALOR ($) ---
    df.loc[df["CATEGORIA"] == "INGRESO EFECTIVO", "CONCEPTO"] = df.loc[df["CATEGORIA"] == "INGRESO EFECTIVO", "INGRESO EFECTIVO"]

    # --- Reglas EFECTIVO INGRESO / EGRESO ---
    def asignar_efectivos(row):
        cat = row["CATEGORIA"]
        val = row["VALOR ($)"]
        efectivo = row["EFECTIVO"]
        if cat in ["BASE DÍA", "INGRESO EFECTIVO"]:
            return pd.Series([val, 0])
        elif cat == "SERVICIOS" and isinstance(efectivo, (int, float)) and efectivo > 0:
            return pd.Series([efectivo, 0])
        elif cat in ["ADELANTOS O PRESTAMOS", "ENTREGA EFECTIVO DIA A", "GASTOS Ò PAGOS PROVEEDORES"]:
            return pd.Series([0, val])
        else:
            return pd.Series([None, None])

    df[["EFECTIVO INGRESO", "EFECTIVO EGRESO"]] = df.apply(asignar_efectivos, axis=1)

    # --- EGRESO POR TRANSFERENCIA ---
    df["EGRESO POR TRANSFERENCIA"] = df["CATEGORIA"] == "TRANSFERENCIAS"
    df["EGRESO POR TRANSFERENCIA"] = df["EGRESO POR TRANSFERENCIA"].map({True: "Si", False: "No"})

    # --- Orden final de columnas ---
    columnas_final = [
        "FECHA", "# ORDEN", "CONCEPTO", "CATEGORIA", "TIPO MOVIMIENTO", "CLIENTE", "CELULAR",
        "VEHICULO", "PLACA", "OPERARIO", "FORMA DE PAGO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO",
        "INGRESO POR TRANSFERENCIA", "VALOR ($)", "BANCO VALOR PAGOS TRANSFERENCIAS", "BANCO INGRESO EFECTIVO",
        "EFECTIVO INGRESO", "EFECTIVO EGRESO", "EGRESO POR TRANSFERENCIA"
    ]

    df = df[[col for col in columnas_final if col in df.columns]]
    return df

# === USO ===
# df = xl("'UNIFICADO original'!A1:W2644", headers=True)
# df_tratada = procesar_mes(df)
# df_tratada
